# Drzewa decyzyjne i lasy losowe

## Dane

Dane zebrano na 83 pacjentach poddanych korekcji chirurgii kręgosłupa. Celem było określenie ważnych czynników ryzyka
na kifozę po operacji. Czynnikami ryzyka są wiek w miesiącach, pierwszy z kręgów i liczby zaangażowanych poziomów.
Przypadki 15 i 28 są usuwane, w związku z tym zostaje 81 przypadków.

Kolumny:
* Age = miesiące
* Start = początek kręgów
* Number = liczba kręgów
* Kyphosis = kifoza, present jeśli jest obecna, absent jeśli nieobecna

Kifoza pogłębiona (inaczej plecy okrągłe) to choroba kręgosłupa, której cechą charakterystyczną jest nadmierne wygięcie kręgosłupa ku tyłowi w odcinku piersiowym i krzyżowym. 

http://www.poradnikzdrowie.pl/zdrowie/kregoslup/kifoza-okragle-plecy-hiperkifoza-przyczyny-objawy-leczenie-i-cwiczenia_33614.html

## Import bibliotek

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Pobranie danych

In [ ]:
df = pd.read_csv('.\\Dane\\kyphosis.csv') # dopisać własną ścieżkę

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

## EDA - eksploracyjna analiza danych

Wykonajmy wykresy rozproszenia dla kombinacji par zmiennych.

In [ ]:
sns.pairplot(df,hue='Kyphosis',palette='Set1')

## Zestaw treningowy i zestaw testowy

Podzielmy dane na zestaw treningowy i zestaw testowy!

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop('Kyphosis',axis=1)
y = df['Kyphosis']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

## Drzewa decyzyjne

Zaczniemy od treningu pojedynczego drzewa decyzyjnego.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtree = DecisionTreeClassifier()

In [ ]:
dtree.fit(X_train,y_train)

## Prognozowanie i ocena

Przeanalizujmy nasze drzewo decyzyjne.

In [ ]:
predictions = dtree.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
print(confusion_matrix(y_test,predictions))

In [ ]:
t1 = pd.DataFrame({'Age':12,'Number':12,'Start':3},
                  index=[0])
#w t1 powinna być zachowana kolejność kolumn, w kolejnych wersjach niezachowanie będzie błędem.

In [ ]:
dtree.predict(t1)[0]

## Wizualizacja drzewa

In [ ]:
from IPython.display import Image  
#from sklearn.externals.six import StringIO  

from io import StringIO

from sklearn.tree import export_graphviz
import pydot 

# uruchomić konsolę jako administrator // lepiej wykorzystać pip
# conda install -c anaconda pydot // dla pip install pydot
# conda install graphviz

# dowload and install: graphviz-2.38.msi
# https://graphviz.gitlab.io/_pages/Download/Download_windows.html

import os
# os.environ["PATH"] += os.pathsep + 'C:/anaconda3/Lib/site-packages/graphviz-2.38/bin'
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin'
# nie trzeba dorzucać wtedy do zmiennch środowiskowych PATH

In [ ]:
# lub bezpośrednio
! pip install graphviz

Kilka przykładów z graphviz

In [ ]:
from graphviz import Digraph

g = Digraph('G', filename='hello.gv')

g.edge('Hello', 'World')

# w notatniku
g

In [ ]:
# w pliku
g.view()

In [ ]:
dot = Digraph(comment='The Round Table')

dot.node('A', 'King Arthur')
dot.node('B', 'Sir Bedevere the Wise')
dot.node('L', 'Sir Lancelot the Brave')

dot.edges(['AB', 'AL'])
dot.edge('B', 'L', constraint='false')

dot

Wracamy do przykładu:

In [ ]:
features = list(df.columns[1:])
features

In [ ]:
dot_data = StringIO()  
export_graphviz(dtree, out_file=dot_data,feature_names=features,filled=True,rounded=True)

graph = pydot.graph_from_dot_data(dot_data.getvalue())  
Image(graph[0].create_png())  

Do pomiaru jednorodności liczby obserwacji wpadających do poszczególnych klas w podgrupach domyślnie jest stosowany wskaźnik Giniego.
Z rysunku możemy odczytać m.in. kryteria podziału zmiennych, liczby obserwacji z każdej klasy reprezentowanej przez poszczególne węzły (value).
Dla naszego przykładu: 'Age':12,'Start':3,'Number':12 wychodzi 1 próbka typu absent. Prawdopodobieństwo wyniku liczymy ilość w danej klasie (odpowiednia pozycja z value) podzielona na ilość wszystkich próbek (samples).


## Losowe lasy

Teraz porównajmy model drzewa decyzyjnego z losowym lasem.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)

In [ ]:
rfc_pred = rfc.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,rfc_pred))

In [ ]:
print(classification_report(y_test,rfc_pred))

In [ ]:
rfc.predict(t1)[0]

Przykład na podstawie [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) - Jake VanderPlas;